<a href="https://colab.research.google.com/github/WetSuiteLeiden/data-collection/blob/master/api_koop_cvdr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose of this notebook

Show how we fetch data from the CVDR repository to be used to create our corresponding datasets

## some API notes (you can skip this)

There currently seem to be approximately 85k regulations (approximately 310k documents if you include all versions).


Standard metadata fields are [declared to be](https://standaarden.overheid.nl/cvdr): 
* **identifier** - CVDR-ID (_including_ version number)

* **title**
* **alternative** (...title)
  * there can be multiple. Present on most but not all records.
  * seems to often be the same as title (VERIFY)

* **subject**
  * there can be multiple. Present on most but not all records.
  * apparently a smallish controlled keyword set, including e.g. 'bestuur en recht', 'financiën en economie', 'maatschappelijke zorg en welzijn', 'openbare orde en veiligheid', 'ruimtelijke ordening, verkeer en vervoer', 'milieu', 'volkshuisvesting en woningbouw', 'onderwijs', 'personeel en organisatie'm 'algemeen'

* **creator**
  * specifically named gemeentes, waterschappen, omgevingsdiensten, etc.
  * the `scheme` attribute mentions what kind it is

* **publisher** (Uitgever)

* **source** (Bron / Oorsprong)
  * there can be multiple
  * seems to be the law this regulation is applying (VERIFY) (grondslag of bevoegdheid)
  * Seems to often refer to something general like the Gemeentewet, Provinciewet, but sometimes more specific things (e.g. Algemene wet bestuursrecht, Archiefwet, Winkeltijdenwet, Participatiewet, Wet op het primair onderwijs, Wet maatschappelijke ondersteuning, Ambtenarenwet, Wet algemene bepalingen omgevingsrecht, Wet ruimtelijke ordening, etc.)
  * the `resourceIdentifier` attribute is a URL, the node text is a description
  * seems a little free-form


* **issued** (Uitgiftedatum)
  * there can be multiple
* modified (Wijzigingsdatum)

* **isRatifiedBy** (Beslisser)
  * there can be multiple
  * seems a smallish controlled keyword set, including values like 'gemeenteraad', 'college van burgemeester en wethouders', 'algemeen bestuur', 'dagelijks bestuur', 'burgemeester', 'gemandateerde functionaris', 'gedeputeerde staten', 'geattribueerde functionaris', 'gedelegeerde functionaris', 'heffingsambtenaar', 'provinciale staten', 'invorderingsambtenaar', 'dijkgraaf', 'deelraad'
  * the `scheme` attribute mentions what kind it is

* **isFormatOf** - seems to refer to the the place it was published (isFormatOf pointing out that this is a (non-authoritative? (VERIFY)) copy of the same information)
  * there can be multiple. Present on most but not all records.
  * mostly specific [Gmb](#glossary_g), [Prb](#glossary_p), [Bgr](#glossary_b), [Wsb](#glossary_w) references, but also includes local newsletters, and a number of unknowns and not-applicables
  * seems to also be filled in _somewhat_ free-form
  * the `resourceIdentifier` attribute is a URL, the node text is a description


* language - seems to just be 'nl'? (VERIFY)

* rights - seems to just be the text 'De tekst in dit document is vrij van auteursrecht en databankrecht' (VERIFY)
  * Present on most but not all records.

* format - actually seems unused? (VERIFY)

* **type** (apparently always 'regeling')
  * the `scheme` attribute mentions what kind it is


But there's a bunch more in the database - that are only present on a subset of records - including:
* spatial - where it applies (often the same as creator?)
* betreft
* onderwerp
* kenmerk
* externeBijlage
* redactioneleToevoeging
* gedelegeerdeRegelgeving
* uitwerkingtredingDatum
* terugwerkendekrachtDatum
* license (rare)
<!-- 

in a sample:
 {'identifier': 19999, 'title': 19999, 'language': 19999, 'type': 19999, 'creator': 19999, 'modified': 19999, 'issued': 19999, 'source': 19999, 
  'inwerkingtredingDatum': 19999, 'betreft': 19999, 'isRatifiedBy': 19999, 'kenmerk': 19999, 
  'isFormatOf': 19962, 'alternative': 19515, 'subject': 19965, 'rights': 19987,  
  'spatial': 16338, 'uitwerkingtredingDatum': 13048, 'gedelegeerdeRegelgeving': 7038, 'redactioneleToevoeging': 12810, 'onderwerp': 5364, 'terugwerkendekrachtDatum': 2355, 'externeBijlage': 1636, 'license': 3})


Contents

regeling/aanhef
regeling/regeling-tekst
regeling/regeling-sluiting
regeling/bijlage (relative URL, absolute-ized relative to https://repository.officiele-overheidspublicaties.nl/)


Documents seem to have references to 
: laws (BWB-ID)
: officielebekendmakingen in prb, gmb, and others

-->


See also:
* https://www.koopoverheid.nl/voor-overheden/gemeenten-provincies-en-waterschappen/cvdr/handleiding-cvdr (documentation for the input app, but explains a lot of useful things in passing)
* https://www.koopoverheid.nl/binaries/koop/documenten/instructies/2017/10/23/cvdr-handleiding-deel-6-deel-6-metadata-xml-schema-en-webservices/IPM_dr_4_0_deel_6-Metadata_XML-schema_Webservices-1.pdf
  * (the webservice it mentions seems to not exist anymore?)


## Fetching

In [1]:
import collections, datetime, random, time, pprint

import wetsuite.helpers.notebook
import wetsuite.helpers.localdata
import wetsuite.datacollect.koop_sru 
import wetsuite.helpers.date
import wetsuite.helpers.etree
import wetsuite.helpers.koop_parse

In [2]:
# store to put downloads into:
cvdr_fetched = wetsuite.helpers.localdata.LocalKV( 'cvdr_fetched.db', str, bytes )

# out of interest  (can take a few seconds once it's large, because get_num_items walks through everything)
#cvdr_fetched.summary(get_num_items=True)

In [3]:
# right now this is simplified to just one 'recent stuff please'   (we previously collected a number of queries to pose)
queries = []

# IF you want to fetch a lot of content, then you probably split many years into shorter spans:  (for reference, there are usually 20 to 250 items per day)
# for from_date, to_date in wetsuite.helpers.date.date_ranges( from_date=datetime.date( 2000, 1, 1 ),  to_date=datetime.date.today(), increment_days=50, strftime_format="%Y-%m-%d" ):
#     queries.append( f'dcterms.modified>={from_date} and dcterms.modified<={to_date}' ) # TODO: check whether there is a better field than modified

# IF you only care to update with recent changes:
#   (note: we treat this as "fetch documents that were mentioned", 
#          not as a "re-fetch things that were changed" )
some_time_ago = datetime.date.today() - datetime.timedelta( days=21 )
queries.append( f'dcterms.modified >= {some_time_ago.strftime("%Y-%m-%d")}' )

print( queries )

['dcterms.modified >= 2024-10-28']


In [ ]:
# Post those queries, fetch any referenced documents we didn't already have
# will not be fast because we are keeping to some basic netiquette and rate limiting (_at all_, not even properly)

sru_cvdr = wetsuite.datacollect.koop_sru.CVDR()

for query in queries:
    print( f'Search: {query}' )
    sru_cvdr.search_retrieve( query ) # purely for the number of records, itself only for the progress bar
    numrecs = sru_cvdr.num_records()
    pbar = wetsuite.helpers.notebook.progress_bar( numrecs, description='fetching' )

    count_cached, count_fetched, count_error = 0, 0, 0

    def cvdr_callback( record_node ):
        ''' Read search result records, pick out the URLs to fetch and fetch them. 
            Is a local function because we count per query, in a slightly weirdly scoped way '''
        #print( wetsuite.helpers.etree.debug_color( record_node ) ) # for later reference, if you want to extract more out of these search records
        global count_cached, count_fetched, count_error

        merged = wetsuite.helpers.koop_parse.cvdr_meta( record_node, flatten=True ) 
        # using flatten is a little creative for something that needs to be a precise value (see cvdr_meta's docstring) but in current use it is valid.
        #pprint.pprint( merged )

        for resource_name, resource_key in ( 
            ('XML',  'publicatieurl_xml'),
            ('HTML', 'publicatieurl_xhtml'),
        ):
            if resource_key not in merged:
                print('SKIP: no %r in %r'%(resource_key, merged))
            else:
                try:
                    _, came_from_cache = wetsuite.helpers.localdata.cached_fetch( cvdr_fetched, merged[ resource_key] ) # we currently care only about the XML it links to
                    if not came_from_cache:
                        count_fetched += 1
                        time.sleep( 2 ) # be somewhat nice to the servers
                    else:
                        count_cached += 1
                # mainly expecting 404, 500
                except ValueError as e:
                    count_error += 1
                    print( "ERROR downloading %s: %s  for %r"%(resource_name, e, merged[resource_key]))
                    time.sleep( 20 ) # be somewhat nicer to the servers

        pbar.value       += 1
        pbar.description  = f'{count_fetched} fetched, {count_cached} cached' # , {count_error} errors

    try:
        sru_cvdr.search_retrieve_many( query, at_a_time=500, up_to=50000, callback=cvdr_callback)

    except ValueError as e:
        count_error += 1
        print( "ERROR querying %s: %s"%(query, e) )
    break

## Creating dataset

We'll spare you the full contents of that store,
because it contains most versions of most things, 
is even more overcomplete than that because of past experiments they have done,
and probably not something you want to fetch fully for the sheer size of it.

Mostly for our own reference, it contains keys that are URLs like:
- https://repository.officiele-overheidspublicaties.nl/CVDR/100078/1/html/100078_1.html
- https://repository.officiele-overheidspublicaties.nl/CVDR/100078/1/xml/100078_1.xml

The values are the according files, as bytestrings.

Right now we care more about parseable data than readable pages,
so we focus on the XML (also in the parsing helper functions), 
but also extract HTML for those that prefer it.
We ignore anything else it might contain.

Also, it seems that KOOP search results expose some variation in the capitalisation, led to duplicate URLs such as: 
- https://repository.officiele-overheidspublicaties.nl/CVDR/100078/1/xml/100078_1.xml
- https://repository.officiele-overheidspublicaties.nl/cvdr/100078/1/xml/100078_1.xml

...so we also ensure we pick just one.

In [5]:
# case insensitive choice.
# (We previously had some tests that their contents are identical, and indeed found no difference)

casededup_xml    = collections.defaultdict(list)  # lowercased version of URL -> actual URLs
casededup_html   = collections.defaultdict(list)  # lowercased version of URL -> actual URLs
ignore_list      = []

unique_xml_urls  = []
unique_html_urls = []

for url in cvdr_fetched:
    if url.endswith('.xml'):
        casededup_xml[ url.lower() ].append( url )
    elif url.endswith('.html'):
        casededup_html[ url.lower() ].append( url )
    else:
        ignore_list.append( url )

for lurl in list(casededup_xml):
     url_list = sorted( casededup_xml[lurl] ) # sorting for some consistency in which one we pick - not necessary, but nice
     unique_xml_urls.append( url_list[0] )

for lurl in list(casededup_html):
     url_list = sorted( casededup_html[lurl] ) 
     unique_html_urls.append( url_list[0] )

# report
print( f"The store had {len(cvdr_fetched)} items, of which {len(ignore_list)} not immediately relevant" )
print( f"  Of the relevant ones, {len( unique_xml_urls )} are XMLs, {len( unique_html_urls )} are (X)HMTLs. " )
print( f"  (so approx %d seem to be case duplicates?)"%(
    len(cvdr_fetched) - ( len( unique_xml_urls ) + len( unique_html_urls ))    
) )
if len(ignore_list)>0:
    print("some URLs are ignored include:")
    for url in random.sample( ignore_list, 10):
        print( f'   {url}' )

The store had 904116 items, of which 0 not immediately relevant
  Of the relevant ones, 295261 are XMLs, 291396 are (X)HMTLs. 
  (so approx 317459 seem to be case duplicates?)


In [ ]:
# Group expressions by their work ID (remember, CVDR works have multiple expressions, e.g. 100078 is a work and 100078_1 its first expression/version)
#   More specifically, we want to create a dict like:
#     work_id -> [ {dict with version, xml_url, html_url}, ... ]
#   We spend some extra code to be able to deal with the absence of html (but not xml)
# ...and _then_ pick just the last

def work_expression_in_url(url):
    # fish IDs out of an URL like 'https://repository.officiele-overheidspublicaties.nl/CVDR/100078/1/xml/100078_1.xml'
    ids                    = url.rsplit('/',1)[1].rsplit('.',1)[0]                # output would e.g. be '100078_1'
    work_id, expression_id = wetsuite.helpers.koop_parse.cvdr_parse_identifier(ids)
    version_int            = int( expression_id.split('_',1)[1], 10)   # as an integer, mainly for correct sorting
    return work_id, expression_id, version_int                         # output would e.g. be ('100078', '100078_1', 1)


group_collect = collections.defaultdict( lambda: collections.defaultdict(dict) ) # workid-> { expressionid: }

for url in unique_xml_urls:
    work_id, expression_id, version_int = work_expression_in_url( url )
    group_collect[work_id][expression_id]['xml']     = url
    group_collect[work_id][expression_id]['version'] = version_int

for url in unique_html_urls:
    work_id, expression_id, version_int = work_expression_in_url( url )
    group_collect[work_id][expression_id]['html'] = url

# now we can actually do that choice of the last from each
lasts_only = {}
for work_id in group_collect:
#for work_id in list(group_collect)[10:11]:
    versions_dict = list( group_collect[work_id].items() )
    choice_key, choice_dict = sorted( versions_dict, key=lambda x:x[1]['version'])[-1] # details of last version
    lasts_only[work_id] = (choice_dict['version'], choice_key, choice_dict.get('xml'), choice_dict.get('html') )

Start a store that intends to contain just the most recent expression XML for each work.
And the same for HTML

In [ ]:
# (may take a few minuets, just to write a few GB of data)
cvdr_latestonly_xml = wetsuite.helpers.localdata.LocalKV( 'cvdr-mostrecent-xml.db', str, bytes )
cvdr_latestonly_xml._put_meta('description_short',  'Raw XML for the latest expression within each CVDR work set')
cvdr_latestonly_xml._put_meta('description',''' ''')

cvdr_latestonly_html = wetsuite.helpers.localdata.LocalKV( 'cvdr-mostrecent-html.db', str, bytes )
cvdr_latestonly_html._put_meta('description_short',  'Raw HTML for the latest expression within each CVDR work set')
cvdr_latestonly_html._put_meta('description',''' ''')

for work_id, (version, expr_id, xml_url, html_url) in wetsuite.helpers.notebook.ProgressBar( lasts_only.items() ):
    cvdr_latestonly_xml.put( work_id, cvdr_fetched.get( xml_url ), commit=False )
    if html_url is not None:
        cvdr_latestonly_html.put( work_id, cvdr_fetched.get( html_url ), commit=False )

cvdr_latestonly_xml.commit()
cvdr_latestonly_html.commit()

In [ ]:
cvdr_latestonly_xml.summary(True)

...and stores that contain the plain text, and the metadata, for the same latest expressions. 

These three stores should have exactly the same keys (unless maybe we forget to clean the lastest leftoves betwen rerunning this).

In [ ]:
cvdr_latestonly_text = wetsuite.helpers.localdata.LocalKV( 'cvdr-mostrecent-text.db', str, str )
cvdr_latestonly_text._put_meta('description_short','Flattened plain text for the latest expression within each CVDR work set') 
cvdr_latestonly_text._put_meta('description',''' ''') 

cvdr_latestonly_meta = wetsuite.helpers.localdata.MsgpackKV( 'cvdr-mostrecent-meta-struc.db', str, None)
cvdr_latestonly_meta._put_meta('description_short','Metadata for the latest expression within each CVDR work set') 
cvdr_latestonly_meta._put_meta('description',''' ''') 


unknown_xml = 0
for work_id, xml_bytes in wetsuite.helpers.notebook.ProgressBar( cvdr_latestonly_xml.items() ):
#for url in wetsuite.helpers.notebook.ProgressBar( list(cvdr_latestonly_xml.keys())[210000:] ):
#        xml_bytes = cvdr_latestonly_xml.get( url )

    tree = wetsuite.helpers.etree.fromstring( xml_bytes )
        
    if work_id not in cvdr_latestonly_meta:
        try:
                meta = wetsuite.helpers.koop_parse.cvdr_meta(tree, flatten=True)
                cvdr_latestonly_meta.put(work_id, meta, commit=False)
        except ValueError as ve: # probably us noticing we don't know a variant of XML
                #print( f'{ve} for {url}' )
                unknown_xml += 1
                #pprint.pprint(meta)

    if work_id not in cvdr_latestonly_text:
        try:
                text = wetsuite.helpers.koop_parse.cvdr_text(tree)
                cvdr_latestonly_text.put(work_id, text, commit=False)
        except AttributeError as ae:
                #print( f'{ae} for {url}' )
                unknown_xml += 1

cvdr_latestonly_meta.commit()
cvdr_latestonly_text.commit()

unknown_xml

In [81]:
# examples of the metadata
cvdr_latestonly_meta.random_sample(3)

[('119373',
  {'identifier': 'CVDR119373_1',
   'title': 'Besluit voorzieningen maatschappelijke ondersteuning gemeente Vlagtwedde 2011',
   'language': 'nl',
   'type': 'regeling (overheid:Informatietype)',
   'creator': 'Vlagtwedde (overheid:Gemeente)',
   'modified': '2017-09-05',
   'spatial': 'Vlagtwedde (overheid:Gemeente)',
   'isFormatOf': 'Ter Apeler Courant d.d. 24 augustus 2011 ()',
   'alternative': 'Besluit voorzieningen maatschappelijke ondersteuning gemeente Vlagtwedde 2011',
   'source': 'Geen ()',
   'isRatifiedBy': 'college van burgemeester en wethouders (overheid:BestuursorgaanGemeente)',
   'subject': 'maatschappelijke zorg en welzijn',
   'issued': '2011-08-16',
   'rights': 'De tekst in dit document is vrij van auteursrecht en\n                    databankrecht',
   'inwerkingtredingDatum': '2011-08-25',
   'uitwerkingtredingDatum': '2011-12-31',
   'betreft': 'Onbekend',
   'kenmerk': 'ZA.11-12573',
   'gedelegeerdeRegelgeving': 'Geen',
   'redactioneleToevoeging